In [ ]:
%pip install langchain langchain_community openai python-dotenv duckduckgo-search  google-api-python-client 

In [3]:
from dotenv import load_dotenv
import os
load_dotenv() 
os.environ['GOOLE_API_KEY'][:5], os.environ['GOOGLE_CSE_ID'][:5]

('AIzaS', 'b3779')

랭체인 에이전트 툴 (검색은 google)

In [27]:
from langchain.agents import initialize_agent, Tool, AgentType
from langchain.llms import OpenAI
from langchain.tools import DuckDuckGoSearchResults
from googleapiclient.discovery import build

# OpenAI LLM 설정
llm = OpenAI(temperature=0.3)



def google_search(query, api_key, cse_id):
    service = build("customsearch", "v1", developerKey=api_key)
    res = service.cse().list(
        q=query,
        cx=cse_id,
        lr='lang_ko',
        hl='ko',
        gl='kr'
    ).execute()
    items = res.get('items', [])
    results_text = []
    for item in items:
        title = item.get('title')
        snippet = item.get('snippet')
        link = item.get('link')
        results_text.append(f"제목: {title}\n요약: {snippet}\n링크: {link}\n")
    return "\n\n".join(results_text)

# Tool 등록 예
from langchain.agents import Tool

tools = [
    Tool(
        name="GoogleSearch",
        func=lambda q: google_search(q, os.environ['GOOLE_API_KEY'], os.environ['GOOGLE_CSE_ID']),
        description=(
            "이 도구는 입력된 한글 쿼리로 구글 한국어 검색을 수행합니다. "
            "검색 결과로 각 문서의 제목, 요약, 링크를 제공합니다. "
            "당신은 이 결과들을 종합해 질문에 대해 상세하고 구체적인 한글 답변을 만들어야 합니다. "
        )
    )
]

# RAG 에이전트 초기화
agent = initialize_agent(
    tools, llm, agent_type=AgentType.ZERO_SHOT_REACT_DESCRIPTION, verbose=True
)

# 사용자의 질문에 대해 응답 처리
prompt_prefix = "아래 검색 결과를 참고하여 한국어로 상세하고 자연스럽게 답변해 주세요.\n"

user_input = "한국의 주요 이슈를 한국어로 자세히 알려줘."
try:
    response = agent.run(prompt_prefix + user_input)
except Exception as e:
    print(f'관련 자료를 찾을수 없습니다. : \n\n{e}')
else:
    print(response)




> Entering new AgentExecutor chain...
 I should use GoogleSearch to find relevant articles about major issues in Korea.
Action: GoogleSearch
Action Input: "한국의 주요 이슈"
Observation: 제목: [지급결제조사자료 2022-3] 암호자산 규제 관련 주요 이슈 및 입법 방향
요약: 한국은행 BANK OF KOREA · 한국은행 · 한국은행이 하는 일 · 설립목적 및 역사 · 한국은행법의 변천 ...
링크: https://www.bok.or.kr/portal/bbs/B0000232/view.do?nttId=10074134&menuNo=200706&pageIndex=1


제목: 멕시코 대선의 주요 이슈와 한국에 주는 시사점
요약: 2024. 5. 29. ... 본고에서는 현재 지지율 1, 2위를 기록 중인 셰인바움 후보와 갈베스 후보의 무역·통상 관련 주요 공약, 미국-멕시코. 간 무역·통상 이슈에 대한 두 후보 ...
링크: https://www.kiep.go.kr/galleryDownload.es?bid=0004&list_no=11314&seq=1


제목: 물적분할과 모자기업 동시상장의 주요 이슈 | | 보고서 | 자본시장연구원
요약: 2022. 6. 2. ... 최근 상장기업의 물적분할과 쪼개기상장이 사회적 이슈로 부상되었으나 대부분의 논의가 일부 대기업 사례에만 국한되어 있다.
링크: https://www.kcmi.re.kr/report/report_view?report_no=1489


제목: 멕시코 대선의 주요 이슈와 한국에 주는 시사점 | KIEP 세계경제 ...
요약: 2024. 5. 29. ... ▷ 미국-멕시코 간 주요 이슈로 △중국의 대(對)미국 우회 수출 증가, △멕시코의 에너지 정책, △USMCA 및 북미 공급망, △이민 등을 꼽을 수 있음.
링크: https://www.kiep.go.kr/gallery.

# 수업코드

## Pinecone 기반 RAG 구현을 위한 환경 구성

In [30]:
%pip install openai pinecone-client langchain langchain-openai langchain_community langchain-pinecone python-dotenv pinecone

Note: you may need to restart the kernel to use updated packages.


In [ ]:
import os
from dotenv import load_dotenv
load_dotenv()  # .env 파일의 환경변수 로드

# Pinecone 연결 및 인덱스 초기화
from pinecone import Pinecone as PineconeClient

# API 키와 환경명 가져오기
pinecone_api_key = os.getenv("PINECONE_API_KEY")
pinecone_env = os.getenv("PINECONE_ENVIRONMENT")

# Pinecone 클라이언트 초기화
pc = PineconeClient(api_key=pinecone_api_key, environment=pinecone_env)

# 사용할 인덱스 이름과 임베딩 차원 설정
index_name = "example-index"
embedding_dim = 1536  # text-embedding-3-small의 벡터 차원

from pinecone import ServerlessSpec

# 인덱스 생성 (없으면 새로 생성, 이미 존재하면 넘어감)
# 연결된 임베딩 모델의 벡터차원과 학습시 사용한 알고리즘(cosine, euclidean, dotproduct) 종류를 맞춰줘야함.. 
if not pc.has_index(index_name):
    pc.create_index(
        name=index_name,
        dimension=embedding_dim,
        metric="cosine",
        spec=ServerlessSpec(
            cloud="aws",
            region="us-east-1"
        )
    )

# 인덱스 객체 연결
index = pc.Index(index_name)
# OpenAI 임베딩 모델 설정 (text-embedding-3-small 사용)
from langchain_openai import OpenAIEmbeddings
embeddings = OpenAIEmbeddings(model="text-embedding-3-small")

## 문서 임베딩

In [8]:
from langchain_core.documents import Document

# 예시 문서 생성 (내용과 메타데이터)
doc1 = Document(page_content="Building an exciting new project with LangChain - come check it out!",
                metadata={"source": "tweet"})
doc2 = Document(page_content="The weather forecast for tomorrow is cloudy and overcast, with a high of 62 degrees.",
                metadata={"source": "news"})
doc3 = Document(page_content="LangGraph is the best framework for building stateful, agentic applications!",
                metadata={"source": "tweet"})

# 벡터스토어 초기화 (PineconeVectorStore에 Pinecone 인덱스와 임베딩 객체 연결)
from langchain_pinecone import PineconeVectorStore
vector_store = PineconeVectorStore(index=index, embedding=embeddings)

# 문서들을 벡터 임베딩하여 Pinecone에 저장
vector_store.add_documents([doc1, doc2, doc3])
# 비동기라 실제반영에 시간이 걸릴수 있음
print(f"현재 벡터 DB 내 벡터 개수: {index.describe_index_stats()['total_vector_count']}")  # 벡터 총량 출력

현재 벡터 DB 내 벡터 개수: 0


In [10]:
print(index.describe_index_stats())

{'dimension': 1536,
 'index_fullness': 0.0,
 'metric': 'cosine',
 'namespaces': {'': {'vector_count': 3}},
 'total_vector_count': 3,
 'vector_type': 'dense'}


In [11]:
print(f"현재 벡터 DB 내 벡터 개수: {index.describe_index_stats()['total_vector_count']}")  # 벡터 총량 출력

현재 벡터 DB 내 벡터 개수: 3


In [12]:
# 검색은 넓게 -> 상위랭크로 재 정럴 하는 전략이 필요해 보임

## 데이터 입력

In [14]:
%pip install pandas

  Using cached pandas-2.2.3-cp312-cp312-win_amd64.whl.metadata (19 kB)
  Using cached tzdata-2025.2-py2.py3-none-any.whl.metadata (1.4 kB)
Using cached pandas-2.2.3-cp312-cp312-win_amd64.whl (11.5 MB)
Using cached tzdata-2025.2-py2.py3-none-any.whl (347 kB)

   ---------------------------------------- 0/2 [tzdata]
   -------------------- ------------------- 1/2 [pandas]
   -------------------- ------------------- 1/2 [pandas]
   -------------------- ------------------- 1/2 [pandas]
   -------------------- ------------------- 1/2 [pandas]
   -------------------- ------------------- 1/2 [pandas]
   -------------------- ------------------- 1/2 [pandas]
   -------------------- ------------------- 1/2 [pandas]
   -------------------- ------------------- 1/2 [pandas]
   -------------------- ------------------- 1/2 [pandas]
   -------------------- ------------------- 1/2 [pandas]
   -------------------- ------------------- 1/2 [pandas]
   -------------------- ------------------- 1/2 [pandas]


In [48]:
import pandas as pd

# 예시를 위한 간단한 데이터프레임 생성 (실제로는 CSV를 로드)
# df = pd.read_csv("KoreanDramas.csv")  # 실제 데이터셋 사용 시
data = [
    {
        "title": "응답하라 1988",
        "year": 2015,
        "genre": ["드라마", "코미디"],
        "director": "신원호",
        "actors": ["혜리", "박보검", "류준열"],
        "rating": 9.2,
        "synopsis": "1988년 서울, 쌍문동 이웃들 사이의 우정과 가족애를 그린 드라마."
    },
    {
        "title": "기생충",
        "year": 2019,
        "genre": ["드라마", "스릴러"],
        "director": "봉준호",
        "actors": ["송강호", "이선균", "조여정"],
        "rating": 8.6,
        "synopsis": "가난한 가족과 부유한 가족 사이 벌어지는 블랙코미디 풍자의 스릴러 영화."
    }
]
df = pd.DataFrame(data)
df.head(2)  # 데이터 확인 (실제로는 전체 데이터 사용)

,title,year,genre,director,actors,rating,synopsis
0,응답하라 1988,2015,"[드라마, 코미디]",신원호,"[혜리, 박보검, 류준열]",9.2,"1988년 서울, 쌍문동 이웃들 사이의 우정과 가족애를 그린 드라마."
1,기생충,2019,"[드라마, 스릴러]",봉준호,"[송강호, 이선균, 조여정]",8.6,가난한 가족과 부유한 가족 사이 벌어지는 블랙코미디 풍자의 스릴러 영화.


In [47]:
import os
from dotenv import load_dotenv
load_dotenv()  # .env 파일의 환경변수 로드

# Pinecone 연결 및 인덱스 초기화
from pinecone import Pinecone as PineconeClient

# API 키와 환경명 가져오기
pinecone_api_key = os.getenv("PINECONE_API_KEY")
pinecone_env = os.getenv("PINECONE_ENVIRONMENT")

# Pinecone 클라이언트 초기화
pc = PineconeClient(api_key=pinecone_api_key, environment=pinecone_env)

# 사용할 인덱스 이름과 임베딩 차원 설정
index_name = "movie-index"
embedding_dim = 1536  # text-embedding-3-small의 벡터 차원

from pinecone import ServerlessSpec

# 인덱스 생성 (없으면 새로 생성, 이미 존재하면 넘어감)
# 연결된 임베딩 모델의 벡터차원과 학습시 사용한 알고리즘(cosine, euclidean, dotproduct) 종류를 맞춰줘야함.. 
if not pc.has_index(index_name):
    pc.create_index(
        name=index_name,
        dimension=embedding_dim,
        metric="cosine",
        spec=ServerlessSpec(
            cloud="aws",
            region="us-east-1"
        )
    )

# 인덱스 객체 연결
index = pc.Index(index_name)
# OpenAI 임베딩 모델 설정 (text-embedding-3-small 사용)
from langchain_openai import OpenAIEmbeddings
embeddings = OpenAIEmbeddings(model="text-embedding-3-small")

In [49]:
index.describe_index_stats()

{'dimension': 1536,
 'index_fullness': 0.0,
 'metric': 'cosine',
 'namespaces': {},
 'total_vector_count': 0,
 'vector_type': 'dense'}

In [50]:
texts= df.synopsis.to_list()
texts

['1988년 서울, 쌍문동 이웃들 사이의 우정과 가족애를 그린 드라마.',
 '가난한 가족과 부유한 가족 사이 벌어지는 블랙코미디 풍자의 스릴러 영화.']

In [51]:
import time
# 벡터스토어 초기화 (PineconeVectorStore에 Pinecone 인덱스와 임베딩 객체 연결)
from langchain_pinecone import PineconeVectorStore
vector_store = PineconeVectorStore(index=index, embedding=embeddings)

docs = []
for i, sample in enumerate(data):
    text = sample['synopsis']            
    del sample['synopsis']
    sample['id'] =f'movie-{i}' 
    docs.append(Document(page_content=text, metadata=sample))
docs    


# docs = [Document(page_content=doc, metadata={"source": "movie"}) for doc in texts]
# # 문서들을 벡터 임베딩하여 Pinecone에 저장
vector_store.add_documents(docs)
# # 비동기라 실제반영에 시간이 걸릴수 있음
# time.sleep(2)
# print(f"현재 벡터 DB 내 벡터 개수: {index.describe_index_stats()['total_vector_count']}")  # 벡터 총량 출력

['5703d0f4-4db1-4023-a8d9-fea78b00d67e',
 'ba4d417b-6dbb-44f0-b534-7ba60911271b']

In [53]:
print(f"현재 벡터 DB 내 벡터 개수: {index.describe_index_stats()['total_vector_count']}")  # 벡터 총량 출력

현재 벡터 DB 내 벡터 개수: 2


## 생성 순서
````
환경 변수 로드
    # .env 파일의 환경변수 로드

벡터 DB (Pinecone) -- openai의 임베딩 모델과 결합하여 모델의 특성에 맞게(유사도,유클리드,닷프로덕트) 설정하여
객체를 생성한다. 
이 벡터는 데이터를 serverless(aws와 같은 클라우드) 기반으로 데이터를 임베딩모델을 이용해서 벡터화하고 유사도 검색 메소드 등 다양한 메소드를 제공하는 클라우드형 벡터DB로 개인사용에 최적화 되어 있다
상세 과정
     Pinecone 임포트
     Pinecone 클라이언트 초기화

     사용할 인덱스 이름과 임베딩 차원 설정

     인덱스 생성 (없으면 새로 생성, 이미 존재하면 넘어감)
     연결된 임베딩 모델의 벡터차원과 학습시 사용한 알고리즘(cosine, euclidean, dotproduct) 종류를 맞춰줘야함.. 
     인덱스 객체 연결
     OpenAI 임베딩 모델 설정 (text-embedding-3-small 사용)

     Document 문서 생성 (내용과 메타데이터)
     벡터스토어 초기화 (PineconeVectorStore에 Pinecone 인덱스와 임베딩 객체 연결)
     문서들을 벡터 임베딩하여 Pinecone에 저장
````

In [59]:
vectors = embeddings.embed_documents([doc.page_content for doc in docs])

In [62]:
len(vectors[0])

1536

# 쿼리 작성후 vector store에서 검색

In [65]:
query = '웃긴 영화가 보고싶어.. 추천 해 주세요'
result = vector_store.similarity_search(query,k=2)
result

[Document(id='5703d0f4-4db1-4023-a8d9-fea78b00d67e', metadata={'actors': ['혜리', '박보검', '류준열'], 'director': '신원호', 'genre': ['드라마', '코미디'], 'id': 'movie-0', 'rating': 9.2, 'title': '응답하라 1988', 'year': 2015.0}, page_content='1988년 서울, 쌍문동 이웃들 사이의 우정과 가족애를 그린 드라마.'),
 Document(id='ba4d417b-6dbb-44f0-b534-7ba60911271b', metadata={'actors': ['송강호', '이선균', '조여정'], 'director': '봉준호', 'genre': ['드라마', '스릴러'], 'id': 'movie-1', 'rating': 8.6, 'title': '기생충', 'year': 2019.0}, page_content='가난한 가족과 부유한 가족 사이 벌어지는 블랙코미디 풍자의 스릴러 영화.')]

# 임의 데이터 생성

In [67]:
import random

# 샘플 데이터 정의
titles = [
    "봄날은 간다", "명량", "살인의 추억", "부산행", "올드보이", "태극기 휘날리며", "추격자", "해운대", 
    "극한직업", "도둑들", "검사외전", "신세계", "범죄와의 전쟁", "관상", "7번방의 선물", "국제시장",
    "비열한 거리", "검은 사제들", "미스터 션샤인", "이태원 클라쓰", "빈센조", "지옥", "지금 우리 학교는", 
    "도깨비", "호텔 델루나", "시그널", "비밀의 숲", "나의 해방일지", "더 글로리", "무빙"
]

genres = ["드라마", "코미디", "스릴러", "로맨스", "액션", "SF", "판타지", "미스터리", "범죄", "공포"]
directors = ["박찬욱", "봉준호", "임권택", "홍상수", "김지운", "윤종빈", "한지민", "이창동", "최동훈", "신원호"]
actors_pool = ["송강호", "이병헌", "전도연", "김혜수", "마동석", "유아인", "박보영", "아이유", "남궁민", "조승우", "김남길", "수지", "정해인", "김태리", "유재석"]

def generate_random_entry():
    return {
        "title": random.choice(titles) + f" {random.randint(1, 100)}",
        "year": random.randint(1990, 2024),
        "genre": random.sample(genres, k=random.randint(1, 3)),
        "director": random.choice(directors),
        "actors": random.sample(actors_pool, k=random.randint(2, 4)),
        "rating": round(random.uniform(6.0, 9.8), 1),        
    }

# 100개의 랜덤 데이터 생성
data = [generate_random_entry() for _ in range(100)]

# 결과 예시 출력 (5개만 보기)
for d in data[:5]:
    print(d)


{'title': '검은 사제들 98', 'year': 1992, 'genre': ['드라마', 'SF', '미스터리'], 'director': '김지운', 'actors': ['마동석', '유재석', '수지'], 'rating': 8.8}
{'title': '도깨비 31', 'year': 2012, 'genre': ['드라마'], 'director': '봉준호', 'actors': ['이병헌', '유아인', '박보영', '남궁민'], 'rating': 8.8}
{'title': '무빙 87', 'year': 2018, 'genre': ['드라마'], 'director': '윤종빈', 'actors': ['남궁민', '조승우'], 'rating': 7.9}
{'title': '해운대 9', 'year': 2013, 'genre': ['로맨스', '스릴러'], 'director': '임권택', 'actors': ['수지', '아이유', '정해인'], 'rating': 9.0}
{'title': '올드보이 77', 'year': 2016, 'genre': ['드라마'], 'director': '임권택', 'actors': ['조승우', '김혜수', '전도연', '박보영'], 'rating': 7.6}


In [ ]:
import os
from dotenv import load_dotenv
load_dotenv()  # .env 파일의 환경변수 로드

# Pinecone 연결 및 인덱스 초기화
from pinecone import Pinecone as PineconeClient

# API 키와 환경명 가져오기
pinecone_api_key = os.getenv("PINECONE_API_KEY")
pinecone_env = os.getenv("PINECONE_ENVIRONMENT")

# Pinecone 클라이언트 초기화
pc = PineconeClient(api_key=pinecone_api_key, environment=pinecone_env)

# 사용할 인덱스 이름과 임베딩 차원 설정
index_name = "movie2-index"
embedding_dim = 1536  # text-embedding-3-small의 벡터 차원

from pinecone import ServerlessSpec

# 인덱스 생성 (없으면 새로 생성, 이미 존재하면 넘어감)
# 연결된 임베딩 모델의 벡터차원과 학습시 사용한 알고리즘(cosine, euclidean, dotproduct) 종류를 맞춰줘야함.. 
if not pc.has_index(index_name):
    pc.create_index(
        name=index_name,
        dimension=embedding_dim,
        metric="cosine",
        spec=ServerlessSpec(
            cloud="aws",
            region="us-east-1"
        )
    )

# 인덱스 객체 연결
index = pc.Index(index_name)
# OpenAI 임베딩 모델 설정 (text-embedding-3-small 사용)
from langchain_openai import OpenAIEmbeddings
embeddings = OpenAIEmbeddings(model="text-embedding-3-small")

# 벡터스토어 초기화 (PineconeVectorStore에 Pinecone 인덱스와 임베딩 객체 연결)
from langchain_pinecone import PineconeVectorStore
vector_store2= PineconeVectorStore(index=index, embedding=embeddings)

docs = []
for i, sample in enumerate(data):    
    sample['id'] =f'movie-{i}' 
    docs.append(Document(page_content=text, metadata=sample))
vector_store2.add_documents(docs)


In [77]:
query = '무서운 영화가 보고싶어.. 추천 해 주세요'
result = vector_store2.similarity_search(query,k=10)
result

[Document(id='8a6f4ba1-e929-4047-8d95-1dac0384e7d9', metadata={'actors': ['송강호', '조승우', '김태리'], 'director': '임권택', 'genre': ['공포', '드라마', '판타지'], 'id': 'movie-17', 'rating': 7.3, 'title': '나의 해방일지 75', 'year': 2002.0}, page_content='가난한 가족과 부유한 가족 사이 벌어지는 블랙코미디 풍자의 스릴러 영화.'),
 Document(id='da7cf323-3334-4456-aa35-625e70516de0', metadata={'actors': ['유재석', '아이유'], 'director': '신원호', 'genre': ['스릴러'], 'id': 'movie-98', 'rating': 8.0, 'title': '해운대 79', 'year': 1997.0}, page_content='가난한 가족과 부유한 가족 사이 벌어지는 블랙코미디 풍자의 스릴러 영화.'),
 Document(id='6e8b22b0-312d-4506-990e-f17fe0e00fca', metadata={'actors': ['남궁민', '유아인'], 'director': '김지운', 'genre': ['SF'], 'id': 'movie-69', 'rating': 9.8, 'title': '범죄와의 전쟁 44', 'year': 2005.0}, page_content='가난한 가족과 부유한 가족 사이 벌어지는 블랙코미디 풍자의 스릴러 영화.'),
 Document(id='78a105b2-314e-4e95-ab67-aba0d7d40334', metadata={'actors': ['유아인', '수지', '이병헌', '남궁민'], 'director': '이창동', 'genre': ['드라마'], 'id': 'movie-73', 'rating': 8.7, 'title': '나의 해방일지 77', 'year': 1997.0}, pa

In [84]:
# 예시 한글 영화 데이터셋 정의
movies = [
    {
        "id": "movie1",
        "title": "7번방의 선물",
        "year": 2013,
        "genre": "드라마",
        "description": "억울한 누명을 쓰고 교도소에 수감된 아빠와 그의 어린 딸의 감동적인 스토리"
    },
    {
        "id": "movie2",
        "title": "미나리",
        "year": 2020,
        "genre": "드라마",
        "description": "한국계 미국인 가족의 따뜻하고 감성적인 성장 이야기"
    },
    {
        "id": "movie3",
        "title": "기생충",
        "year": 2019,
        "genre": "드라마",
        "description": "가난한 가족과 부자 가족 사이의 빈부격차를 그린 사회 풍자 드라마"
    },
    {
        "id": "movie4",
        "title": "범죄도시",
        "year": 2017,
        "genre": "범죄",
        "description": "형사가 범죄 조직을 소탕하는 범죄 액션 영화"
    },
    {
        "id": "movie5",
        "title": "범죄도시 2",
        "year": 2022,
        "genre": "범죄",
        "description": "형사와 범죄 조직의 대결을 그린 범죄 액션 영화의 속편"
    },
    {
        "id": "movie6",
        "title": "헤어질 결심",
        "year": 2022,
        "genre": "범죄",
        "description": "산에서 발생한 의문의 죽음(살인 사건)을 수사하던 형사가 피의자에게 이끌리며 벌어지는 미스터리 멜로 영화"
    },
    {
        "id": "movie7",
        "title": "다만 악에서 구하소서",
        "year": 2020,
        "genre": "범죄",
        "description": "청부 살인업자와 범죄 조직의 마지막 거래를 그린 범죄 액션 영화"
    }
]

In [85]:
from langchain.embeddings import OpenAIEmbeddings

# OpenAIEmbeddings 객체 생성 (모델명과 API 키 지정)
embeddings = OpenAIEmbeddings(model="text-embedding-3-small", openai_api_key=os.environ["OPENAI_API_KEY"])

# 모든 영화 설명에 대해 임베딩 벡터 생성
descriptions = [movie["description"] for movie in movies]        # 설명문 리스트
movie_vectors = embeddings.embed_documents(descriptions)         # 각 설명문에 대한 임베딩 벡터 리스트 생성

# 임베딩 벡터의 차원 확인 (예상: 1536차원)
print(f"임베딩 벡터 차원: {len(movie_vectors[0])}")

C:\Users\511-17\AppData\Local\Temp\ipykernel_9052\3303327196.py:4: LangChainDeprecationWarning: The class `OpenAIEmbeddings` was deprecated in LangChain 0.0.9 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-openai package and should be used instead. To use it run `pip install -U :class:`~langchain-openai` and import as `from :class:`~langchain_openai import OpenAIEmbeddings``.
  embeddings = OpenAIEmbeddings(model="text-embedding-3-small", openai_api_key=os.environ["OPENAI_API_KEY"])


임베딩 벡터 차원: 1536


In [90]:
# API 키와 환경명 가져오기
pinecone_api_key = os.getenv("PINECONE_API_KEY")
pinecone_env = os.getenv("PINECONE_ENVIRONMENT")

# Pinecone 클라이언트 초기화
pc = PineconeClient(api_key=pinecone_api_key, environment=pinecone_env)
index_name = "movie-vector-index"
embedding_dim = 1536
# Pinecone 초기화 (API 키와 환경은 이미 os.environ에 설정됨)
if not pc.has_index(index_name):
    pc.create_index(
        name=index_name,
        dimension=embedding_dim,
        metric="cosine",
        spec=ServerlessSpec(
            cloud="aws",
            region="us-east-1"
        )
    )

# 인덱스 이름과 차원 설정
index_name = "movie-vector-index"

# 기존에 동일 이름의 인덱스가 있으면 삭제 (재실행 시 중복 방지용)
if index_name in pc.list_indexes():
    pc.delete_index(index_name)

# 인덱스 객체에 연결
index = pc.Index(index_name)

In [91]:
# Pinecone에 벡터 업로드 (upsert)
# 각 레코드는 (id, 벡터, metadata)의 형태로 준비
vector_data = []
for movie, vector in zip(movies, movie_vectors):
    # metadata로 title, genre, year, description 저장
    meta = {
        "title": movie["title"],
        "genre": movie["genre"],
        "year": movie["year"],
        "description": movie["description"]
    }
    vector_data.append((movie["id"], vector, meta))

# 벡터들을 Pinecone 인덱스에 업서트
index.upsert(vectors=vector_data)

{'upserted_count': 7}

In [92]:
# 검색 쿼리 예시
query_text = "감성적인 드라마 영화 추천해줘"

# 쿼리 문장을 임베딩 벡터로 변환
query_vector = embeddings.embed_query(query_text)

# Pinecone에서 벡터 유사도 검색 수행 (코사인 유사도 기반)
# 상위 3개의 가장 가까운 벡터를 찾고, 메타데이터를 포함하여 반환
result = index.query(vector=query_vector, top_k=3, include_metadata=True)

# 결과 출력: 각 결과의 제목, 연도, 장르를 표시
for match in result["matches"]:
    info = match["metadata"]
    print(f"{info['title']} - {info['year']} ({info['genre']})")

기생충 - 2019.0 (드라마)
7번방의 선물 - 2013.0 (드라마)
범죄도시 - 2017.0 (범죄)


In [93]:
# 메타데이터 필터를 활용한 검색: 2020년 이후 개봉한 영화들 중 상위 3개 반환
query_text2 = "영화"  # 매우 일반적인 쿼리
query_vector2 = embeddings.embed_query(query_text2)

# year 필터 적용 (year >= 2020인 항목만 대상)
filter_condition = {"year": {"$gte": 2020}}

result2 = index.query(vector=query_vector2, top_k=3, filter=filter_condition, include_metadata=True)

for match in result2["matches"]:
    info = match["metadata"]
    print(f"{info['title']} - {info['year']} ({info['genre']})")

범죄도시 2 - 2022.0 (범죄)
헤어질 결심 - 2022.0 (범죄)
다만 악에서 구하소서 - 2020.0 (범죄)


#  - 정리 - 
```
기본환경설정
데이터 수집
벡터DB 선택
쿼리 작성 및 결과 확인
```

In [94]:
index_name = 'wiki'
demention = 1536
metric = 'cosine'

In [95]:
%pip install datasets

  Using cached filelock-3.18.0-py3-none-any.whl.metadata (2.9 kB)
  Using cached dill-0.3.8-py3-none-any.whl.metadata (10 kB)
  Using cached xxhash-3.5.0-cp312-cp312-win_amd64.whl.metadata (13 kB)
  Using cached multiprocess-0.70.16-py312-none-any.whl.metadata (7.2 kB)
Using cached dill-0.3.8-py3-none-any.whl (116 kB)
Using cached multiprocess-0.70.16-py312-none-any.whl (146 kB)
   ---------------------------------------- 0.0/25.7 MB ? eta -:--:--
   --------------------- ------------------ 13.6/25.7 MB 71.1 MB/s eta 0:00:01
   --------------------- ------------------ 13.6/25.7 MB 71.1 MB/s eta 0:00:01
   ---------------------------------------  25.4/25.7 MB 42.4 MB/s eta 0:00:01
   ---------------------------------------- 25.7/25.7 MB 39.6 MB/s eta 0:00:00
Using cached filelock-3.18.0-py3-none-any.whl (16 kB)
Using cached xxhash-3.5.0-cp312-cp312-win_amd64.whl (30 kB)

   ----- ---------------------------------- 1/8 [pyarrow]
   ----- ---------------------------------- 1/8 [pyarrow]
 

In [116]:
#  데이터 수집집
from datasets import load_dataset
data = load_dataset('wikipedia','20220301.simple',split='train[:100]')

In [115]:
data

Dataset({
    features: ['id', 'url', 'title', 'text'],
    num_rows: 100
})

In [ ]:
# API 키와 환경명 가져오기
pinecone_api_key = os.getenv("PINECONE_API_KEY")
pinecone_env = os.getenv("PINECONE_ENVIRONMENT")

# Pinecone 클라이언트 초기화
pc = PineconeClient(api_key=pinecone_api_key, environment=pinecone_env)
index_name = "movie-vector-index"
embedding_dim = 1536
# Pinecone 초기화 (API 키와 환경은 이미 os.environ에 설정됨)
if not pc.has_index(index_name):
    pc.create_index(
        name=index_name,
        dimension=embedding_dim,
        metric="cosine",
        spec=ServerlessSpec(
            cloud="aws",
            region="us-east-1"
        )
    )

# 인덱스 이름과 차원 설정
index_name = "movie-vector-index"

# 기존에 동일 이름의 인덱스가 있으면 삭제 (재실행 시 중복 방지용)
if index_name in pc.list_indexes():
    pc.delete_index(index_name)

# 인덱스 객체에 연결
index = pc.Index(index_name)

##################################################
from langchain.embeddings import OpenAIEmbeddings

# OpenAIEmbeddings 객체 생성 (모델명과 API 키 지정)
embeddings = OpenAIEmbeddings(model="text-embedding-3-small", openai_api_key=os.environ["OPENAI_API_KEY"])

from langchain_text_splitters import RecursiveCharacterTextSplitter
text_splitter = RecursiveCharacterTextSplitter(chunk_size=400, chunk_overlap=20,separators=['\n\n','\n'])
texts,metas,batch_size = [],[],100
for i ,sample in enumerate(data):
    text = sample.pop('text')
    chunk_text = text_splitter.split_text(text)    
    for i, chunk in enumerate(chunk_text):
        record = {
            'chunk_id' : i,
            'text' : text,
            **sample
        }
        texts.append(chunk_text)
        metas.append(record)
        if (i+1) % batch_size == 0:
            vectors = embeddings.embed_documents(texts)

            # 벡터화
            # index에 upset
# 미 완성......................    